In [99]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os


# Any results you write to the current directory are saved as output.

sd_traffic = pd.read_csv("../Hackathon Datasets/Traffic/sd_traffic_2015.csv")
sd_traffic_stations = pd.read_csv("../Hackathon Datasets/Traffic/dot_traffic_stations_2015.txt")

In this notebook we will be exploring the US Traffic 2015 dataset for San Diego. The dataset was taken from the US department of Transportation.

In [100]:
dof = sd_traffic_stations["lane_of_travel"].unique()

sd_station_ids = sd_traffic_stations["station_id"].unique()

#Creating a mapping of the station_ids to latitude and longitude location stored in a list
station_mappings_latitude = {}
station_mappings_longitude = {}

#iterates through each station id in the file to perform the mapping 
for i in sd_station_ids:
    station_mappings_latitude[i] = pd.unique(\
                                              sd_traffic_stations[sd_traffic_stations["station_id"] == i]\
                                              ["latitude"])
    station_mappings_longitude[i] = pd.unique(\
                                               sd_traffic_stations[sd_traffic_stations["station_id"] == i]\
                                               ["longitude"])
    
print(sd_traffic.shape)
print(sd_traffic_stations.shape)

(191057, 39)
(28466, 55)


Description of data. Below will have additional inofmration regarding the type of data to better describe the graphs. Some information could not be found which will be highlighted in italics

San Diego Traffic dataset (191057x39 dataframe)
    * date
    * day_of_data
    * day_of_week
    * fips_state_code: Federal Information Processing Standards. 06 assigned to California
    * <i>lane_of_travel</i>
    * month_of_data
    * <i>record_type</i>
    * restrictions: All data is NaN
    * <i>station_id</i>
    * traffic_volume_counted_after_.... - Traffic volume by bins 
    * year_of_data: Year the data was taken 
    * Direction of Travel:
        1 - North
        3- East
        5 - South
        7 - West
    * Functional Classification: Symbol
    * Functional Classification Name: Type of road (urban: principal arterial - interstate)

San Diego Traffic Station dataset (52x56 dataframe)
    * algorithm_of_vehicle_classification: All values are NaN
    * algorithm_of_vehicle_classification_name: All values are NaN
    * calibration_of_weighing_system
    * calibration_of_weighing_system_name
    * classification_system_for_vehicle_classification
    * concurrent_route_signing
    * concurrent_signed_route_number
    * concurrent_signed_route_number
    * direction_of_travel_name
    * fips_county_code
    * fips_state_code
    * functional_classification
    * functional_classification_name
    * hpms_sample_identifies: Values are NaN
    * hpms_sample_type: Values are NaN
    * lane_of_travel: 0, 1, 2, 3, 4, 5, 7
    * lane_of_travel_name:
        * 0 - data with lanes combined
        * 1 - Outside (rightmost) lane
        * 2 - Other lanes
        * 3 - Other lanes
        * 4 - Other lanes
        * 5 - Other lanes
        * 7 - Other lanes
    * latitude
    * longitude 
    * lrs_identification
    * lrs_location_point
    * method_of_data_retrieval
    * method_of_data_retrieval_name
    * method_of_traffic_volume_counting
    * method_of_traffic_volume_counting_name
    * method_of_truck_weighing 
    * method_of_truck_weighing_name
    * method_of_vehicle_classification 
    * method_of_vehicle_classification_name
    * national_highway_system 
    * number_of_lanes_in_direction_indicated 
    * number_of_lanes_monitored_for_traffic_volume 
    * number_of_lanes_monitored_for_truck_weight 
    * number_of_lanes_monitored_for_vehicle_class
    * posted_route_signing 
    * posted_signed_route_number
    * previous_station_idprimary_purpose 
    * primary_purpose_namerecord_type
    * sample_type_for_traffic_volume 
    * sample_type_for_traffic_volume_name
    * sample_type_for_truck_weight 
    * sample_type_for_truck_weight_name 
    * sample_type_for_vehicle_classification 
    * sample_type_for_vehicle_classification_name
    * second_type_of_sensor 
    * shrp_site_identificationstation_id
    * station_location 
    * type_of_sensortype_of_sensor_name
    * year_of_data 
    * year_station_discontinued
    * year_station_established
  
We will drop the following columns from the sdt dataset, as of right now we only care about traffic density and not direction of traffic. After this sdt will have 33 columns
    * fips_state_code, lane_of_travel, record_type, restrictions, functional_classification,
    functional_classification_name




In [101]:
sd_traffic_drop_columns = ["fips_state_code","lane_of_travel","record_type","restrictions", \
                   "functional_classification","functional_classification_name"]
sd_traffic_ds = sd_traffic.drop(sd_traffic_drop_columns, axis=1)
sd_traffic_ds.shape

(191057, 33)

We will now add the latitude and longitude for all the stations ids found in sd_traffic. 

In [102]:

#Because we were originally looking at US data some states of the same station IDs
#Because they have the same station IDs they will also have multiple latitude and longitude
#points

lat_north = 32.820891
lat_south = 32.674709
long_west = 117.255457 #Value includes negative but dataset has removed this
long_east = 117.017824 #Value includes negative but dataset has removed this

extra_latitude = {}
extra_longitude = {}
test = sd_traffic_stations[['station_id','latitude','longitude']].copy()
#for j in station_mappings_latitude.keys():
#    if len(station_mappings_latitude[j]) > 1:
#        extra_latitude[j] = station_mappings_latitude[j]

#for k in station_mappings_longitude.keys():
#    if len(station_mappings_longitude[k]) > 1:
#        extra_longitude[k] = station_mappings_longitude[k]

sd_traffic["Latitude"] = 0
sd_traffic["Longitude"] = 0
for i in station_mappings_latitude.keys():
    if i in sd_traffic["station_id"]:
        print("station id",i,"is in sd_traffic")
        if station_mappings_latitude[i] > 1:
            print("station_mappings_latitude value is greater than 1")
            for item in station_mappings_latitude[i]:
                if item <= lat_north & item >= lat_south:
                    sd_traffic[sd_traffic["station_id"] == i]["Latitude"] = item
        else:
            sd_traffic["Latitude"] = np.where(sd_traffic["station_id"] == i, \
                                      station_mappings_latitude[i],"None")
    
#for j in station_mappings_longitude.keys():
#    sd_traffic["Longitude"] = np.where(sd_traffic["station_id"] == i, \
#                                       station_mappings_longitude[i],"None")



In [104]:
#print(sd_traffic[sd_traffic["station_id"] == station_mappings_latitude.keys()])
sd_traffic.columns

Index(['Unnamed: 0', 'date', 'day_of_data', 'day_of_week',
       'direction_of_travel', 'direction_of_travel_name', 'fips_state_code',
       'functional_classification', 'functional_classification_name',
       'lane_of_travel', 'month_of_data', 'record_type', 'restrictions',
       'station_id', 'traffic_volume_counted_after_0000_to_0100',
       'traffic_volume_counted_after_0100_to_0200',
       'traffic_volume_counted_after_0200_to_0300',
       'traffic_volume_counted_after_0300_to_0400',
       'traffic_volume_counted_after_0400_to_0500',
       'traffic_volume_counted_after_0500_to_0600',
       'traffic_volume_counted_after_0600_to_0700',
       'traffic_volume_counted_after_0700_to_0800',
       'traffic_volume_counted_after_0800_to_0900',
       'traffic_volume_counted_after_0900_to_1000',
       'traffic_volume_counted_after_1000_to_1100',
       'traffic_volume_counted_after_1100_to_1200',
       'traffic_volume_counted_after_1200_to_1300',
       'traffic_volume_counted_